<a href="https://colab.research.google.com/github/KaviyaVelusamy/INTEL_UNNATI/blob/main/Running_GenAI_on_Intel_AI_Laptops_and_Simple_LLM_Inference_on_CPU_and_fine_tuning_of_LLM_Models_using_Intel%C2%AE%C2%A0OpenVINO%E2%84%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q torch datasets
!pip install -q accelerate peft bitsandbytes trl
!pip install langchain playwright html2text sentence-transformers faiss-gpu nest_asyncio
!pip install openvino-dev[onnx]


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
from openvino.runtime import Core
import numpy as np


In [ ]:
model_name = 'gpt2'
model = AutoModelForCausalLM.from_pretrained(model_name)
model.eval()


In [ ]:
dummy_input = torch.randint(50257, (1, 64))
torch.onnx.export(model, dummy_input, "model.onnx", opset_version=11, input_names=["input_ids"], output_names=["output"])

In [ ]:
import os
os.system("mo --input_model model.onnx --output_dir model_ir")

In [ ]:
ie = Core()
model_ir = ie.read_model(model="model_ir/model.xml")
compiled_model = ie.compile_model(model=model_ir, device_name="CPU")

In [ ]:
infer_request = compiled_model.create_infer_request()

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
!pip install openvino-dev
!pip install transformers

import openvino as ov #OpenVINO Python API
from transformers import AutoTokenizer
import numpy as np



infer_request = compiled_model.create_infer_request()

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def run_inference(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding="max_length", truncation=True, max_length=64) # Pad/truncate to match model input
    input_ids = inputs["input_ids"].numpy()
    input_tensor = ov.Tensor(input_ids.astype(np.int64)) #OpenVINO Tensor
    infer_request.set_input_tensor(input_tensor) #OpenVINO Tensor to set_input_tensor
    infer_request.infer()
    output = infer_request.get_output_tensor(0)
    output_ids = np.argmax(output.data, axis=-1)
    decoded_output = tokenizer.decode(output_ids[0])
    return decoded_output

In [ ]:
print(run_inference("Who is the principal of GCT college Coimbatore in 2022?"))
print(run_inference("What is the full form for GCT?"))

 is the best of the.??"-batore? the?

 principal principal principal the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the The The The The S S S S - - - - - -
 is the best list of this-?
 form the form form form form form form the the the the the the the the the the the the the the the the the the the the the the the the ( ( ( ( ( ( ( ( ( ( ( ( ( ( ( ( ( ( ( ( ( ( (


In [ ]:
import locale
locale.setlocale(locale.LC_ALL, '')
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install -U langchain-community
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import Html2TextTransformer
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.0 MB/s eta 0:00:00


In [ ]:
!playwright install
nest_asyncio.apply()

articles = ["https://en.wikipedia.org/wiki/Government_College_of_Technology,_Coimbatore",
            "https://gct.ac.in/succession-list",
            "https://en.wikipedia.org/wiki/Gopalswamy_Doraiswamy_Naidu"]

loader = AsyncChromiumLoader(articles)
docs = loader.load()

161.3 MiB [] 0% 0.0s161.3 MiB [] 0% 27.7s161.3 MiB [] 0% 18.4s161.3 MiB [] 0% 12.4s161.3 MiB [] 0% 10.0s161.3 MiB [] 0% 9.5s161.3 MiB [] 1% 8.4s161.3 MiB [] 1% 7.8s161.3 MiB [] 1% 7.0s161.3 MiB [] 2% 6.4s161.3 MiB [] 2% 6.5s161.3 MiB [] 2% 6.6s161.3 MiB [] 3% 6.9s161.3 MiB [] 3% 7.2s161.3 MiB [] 3% 7.4s161.3 MiB [] 3% 7.1s161.3 MiB [] 4% 7.3s161.3 MiB [] 4% 7.4s161.3 MiB [] 4% 7.5s161.3 MiB [] 4% 7.3s161.3 MiB [] 4% 7.1s161.3 MiB [] 5% 7.2s161.3 MiB [] 5% 7.1s161.3 MiB [] 5% 6.9s161.3 MiB [] 5% 6.8s161.3 MiB [] 6% 6.9s161.3 MiB [] 6% 6.8s161.3 MiB [] 7% 6.9s161.3 MiB [] 7% 6.7s161.3 MiB [] 7% 6.6s161.3 MiB [] 8% 6.4s161.3 MiB [] 8% 6.3s161.3 MiB [] 9% 6.1s161.3 MiB [] 9% 5.8s161.3 MiB [] 10% 5.7s161.3 MiB [] 10% 5.6s161.3 MiB [] 11% 5.6s161.3 MiB [] 11% 5.5s161.3 MiB [] 11% 5.4s161.3 MiB [] 12% 5.4s161.3 MiB [] 12% 5.3s161.3 MiB [] 13% 5.4s161.3 MiB [] 14% 5.3s161.3 MiB [] 15% 5.2s161.3 MiB [] 16% 5.2s161.3 MiB [] 16% 5.1s161.3 MiB [] 17% 4.9s161.3 MiB [] 18% 4.7s161.3 MiB [] 18% 4.6s1

In [ ]:
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed)

In [ ]:
db = FAISS.from_documents(chunked_documents, HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

retriever = db.as_retriever(search_type="similarity", search_kwargs={'k': 4})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query = "Principal?"
docs = db.similarity_search(query)
for doc in docs:
    print(doc.page_content)
    print("-------------")

# Succession list of principal

Dr.P.Thamarai

2018 - 2023

Dr.Esther Annlin Kala Jame

2017 – 2018
-------------
* ‎ 
  * My GCT
  * Access GCT E-mail
  * Alumni
  * Contact
-------------
* Mandatory Disclosure
  * Faculty/Staff forms
  * NAAC Forms
  * Internal Audit Forms
  * College Calendar
  * IT Forms
  * Wi-Fi Forms
-------------
## Notable alumni[edit]
-------------


In [ ]:
query = "Arthur Hope?"
docs = db.similarity_search(query)
for doc in docs:
    print(doc.page_content)
    print("-------------")

Through Naidu's efforts and donations India's first polytechnic college, the
Arthur Hope Polytechnic and the Arthur Hope College of Engineering were set
up. Later the college moved to its present location and is now known as
Government College of Technology (GCT). The college was named after the then
Madras governor Arthur Hope. In 1945, GD Naidu was the principal of the
college.[10] Naidu was not satisfied with the four-year programs and said that
it was a waste of time for students. He suggested that two years was more than
enough to teach the same courses, even with different concentrations. However,
the British government did not accept his idea and Naidu resigned from his
post.[11] The name Hope College for the original site of the college remains
in Coimbatore even now.
-------------
His biography _Appa_ was written by the popular Tamil writer Sivasankari.[13]
-------------
## Notable alumni[edit]
-------------
* Mylswamy Annadurai, Former Director ISRO Satellite Centre, Padma Sh